In [1]:
"""loop through files in S1_Dataset directory"""
"""create 2D data matrix by appending each new dataset to the bottom of data matrixx"""
import os
import pandas as pd
S1_PATH = os.path.join('..','..','Datasets_Healthy_Older_People','S1_Dataset')
S2_PATH = os.path.join('..','..','Datasets_Healthy_Older_People','S2_Dataset')
print('Importing Data...')
s1_data = pd.DataFrame()
for filename in os.listdir(S1_PATH):
    if filename != 'README.txt':
        data_path = os.path.join(S1_PATH, filename)
        data=pd.read_csv(data_path, header=None)
        s1_data = s1_data.append(data, ignore_index=True)
s1_data.columns = ['time','frontal','vertical','lateral','id','rssi','phase','frequency','activity']
print('Done')
s1_data.info()
s1_data.tail()

Importing Data...
Done
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52482 entries, 0 to 52481
Data columns (total 9 columns):
time         52482 non-null float64
frontal      52482 non-null float64
vertical     52482 non-null float64
lateral      52482 non-null float64
id           52482 non-null int64
rssi         52482 non-null float64
phase        52482 non-null float64
frequency    52482 non-null float64
activity     52482 non-null int64
dtypes: float64(7), int64(2)
memory usage: 3.6 MB


,time,frontal,vertical,lateral,id,rssi,phase,frequency,activity
52477,485.00,0.330660,0.95081,-0.230340,1,-57.0,6.2188,921.75,1
52478,485.08,0.330660,0.95081,-0.230340,3,-58.0,0.8805,921.75,1
52479,485.50,0.330660,0.95081,-0.230340,1,-60.5,5.0054,924.75,1
52480,488.25,0.119600,0.82449,0.009122,4,-56.0,4.9087,925.75,4
52481,488.75,0.072698,1.31830,0.009122,4,-56.5,1.0370,922.75,4


In [3]:
"""split the data into training and test"""
from sklearn.model_selection import train_test_split
s1_train_set, s1_test_set = train_test_split(s1_data, test_size = 0.2, random_state = 1)
#s1_train_set.info()
#s1_train_set.head()

In [4]:
"""for this task (stage 1 SVM), keep only the acclerometer features"""
s1_train_set_svm =  s1_train_set.drop(columns=['time','id','rssi','phase','frequency'])
s1_train_set_svm_features = s1_train_set_svm.drop(columns=['activity'])
s1_train_set_svm_labels = s1_train_set_svm.drop(columns=['frontal','vertical','lateral'])

In [16]:
"""Train with SVM"""
from sklearn import svm
svm_clf = svm.SVC(C=0.05)
svm_clf.fit(s1_train_set_svm_features,s1_train_set_svm_labels) 

c:\users\denise\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\denise\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=0.05, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [18]:
"""evaluate the trained model using cross validation and the confusion matrix"""
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
s1_train_set_svm_predict = cross_val_predict(svm_clf, s1_train_set_svm_features, s1_train_set_svm_labels, cv=3)
svm_conf_mx = confusion_matrix(s1_train_set_svm_labels, s1_train_set_svm_predict)
svm_conf_mx

c:\users\denise\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\denise\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\denise\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\denise\appdata\local\programs\python\python37\lib\site-packages\

array([[11928,     0,   183,     0],
       [ 3496,     0,     5,     0],
       [   10,     0, 24799,     0],
       [ 1551,     0,    13,     0]], dtype=int64)

In [13]:
"""evaluate the trained model in terms of precision and recall"""
"""1: sit on bed, 2: sit on chair, 3: lying, 4: ambulating"""
for index, activity in [(0, "sit on bed"), (1, "sit on chair"), (2, "lying"), (3, "ambulating")]:
    precision = svm_conf_mx[index,index]/sum(svm_conf_mx[:,index])
    recall = svm_conf_mx[index,index]/sum(svm_conf_mx[index,:])
    print(activity + " precision: " + str(precision))
    print(activity + " recall: " + str(recall))

sit on bed precision: 0.7167389732465654
sit on bed recall: 0.9821649739905871
sit on chair precision: 0.6829268292682927
sit on chair recall: 0.10397029420165667
lying precision: 0.9974652987326493
lying recall: 0.9993147648030957
ambulating precision: 0.0
ambulating recall: 0.0
